In [1]:
import open_clip
open_clip.list_pretrained()

[('RN50', 'openai'),
 ('RN50', 'yfcc15m'),
 ('RN50', 'cc12m'),
 ('RN50-quickgelu', 'openai'),
 ('RN50-quickgelu', 'yfcc15m'),
 ('RN50-quickgelu', 'cc12m'),
 ('RN101', 'openai'),
 ('RN101', 'yfcc15m'),
 ('RN101-quickgelu', 'openai'),
 ('RN101-quickgelu', 'yfcc15m'),
 ('RN50x4', 'openai'),
 ('RN50x16', 'openai'),
 ('RN50x64', 'openai'),
 ('ViT-B-32', 'openai'),
 ('ViT-B-32', 'laion400m_e31'),
 ('ViT-B-32', 'laion400m_e32'),
 ('ViT-B-32', 'laion2b_e16'),
 ('ViT-B-32', 'laion2b_s34b_b79k'),
 ('ViT-B-32', 'datacomp_xl_s13b_b90k'),
 ('ViT-B-32', 'datacomp_m_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_clip_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_laion_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_image_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_text_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_basic_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_s128m_b4k'),
 ('ViT-B-32', 'datacomp_s_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_clip_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_laion_s13m_b4k'),
 ('ViT-B-32', 'commonpool_

In [2]:
import torch
tensor = torch.load('/home/user/data/cc12m_dr/00000/000000000.pth')

In [3]:
tensor

{'image_emb': [tensor([ 0.1157,  1.2656, -0.2930,  ..., -0.4492, -0.3359,  1.3516],
         dtype=torch.bfloat16),
  tensor([0.0024, 1.0938, 0.7031,  ..., 0.2285, 0.5234, 0.6055],
         dtype=torch.bfloat16),
  tensor([-0.1738,  1.5781,  0.3359,  ..., -0.0427,  0.5664,  0.6289],
         dtype=torch.bfloat16),
  tensor([ 0.4258,  0.8086, -0.2412,  ..., -0.0287, -0.5000,  0.4570],
         dtype=torch.bfloat16),
  tensor([ 0.2910,  0.7891,  0.1035,  ..., -0.6445,  0.1064,  0.9688],
         dtype=torch.bfloat16),
  tensor([ 0.1299,  1.3438,  0.0114,  ..., -0.5625,  0.6484,  1.0078],
         dtype=torch.bfloat16),
  tensor([ 0.0298,  0.9766, -0.1768,  ..., -0.1504, -0.4902,  1.3047],
         dtype=torch.bfloat16),
  tensor([ 0.1445,  0.8555,  0.0933,  ..., -0.2676, -0.5352,  0.6680],
         dtype=torch.bfloat16),
  tensor([-0.3789,  1.3594,  0.3340,  ..., -0.2559, -0.6367,  1.0703],
         dtype=torch.bfloat16),
  tensor([ 0.3398,  1.1953, -0.3281,  ..., -0.1572, -0.1943,  1.53

In [5]:
len(tensor['image_emb'])

30

In [6]:
tensor['image_emb'][0].shape

torch.Size([1536])

In [7]:
len(tensor['text_emb'])

4

In [8]:
tensor['text_emb'][0].shape

torch.Size([1536])

In [1]:
import json
with open('/home/user/data/DataCompDR/DataCompDR-12M-bf16/00000000/5252820736bc680b29c6dca972776dc3.paug.json','r') as f:
    dic = json.load(f)
    

In [2]:
print(len(dic['param_aug']))

30


In [3]:
print(len(dic['param_aug'][0]))

2


In [4]:
print(dic['param_aug'][0])

[[33, 29, 48, 57], [[13, 0.0], [11, 178.5]]]


In [14]:
import gzip
import torch

with gzip.open('/home/user/data/DataCompDR/DataCompDR-12M-bf16/00000000/5252820736bc680b29c6dca972776dc3.pth.gz', 'rb') as f:
    tensor = torch.load(f)

In [15]:
print(len(tensor['text_emb']))

7


In [7]:
print(tensor['text_emb'][0].shape)

torch.Size([1536])


In [8]:
print(tensor['image_emb'][0].shape)

torch.Size([1536])


In [9]:
tensor['text_emb']

tensor([[ 0.0037, -0.0552, -0.0244,  ..., -0.0537, -0.0037,  0.0378],
        [ 0.0037, -0.0552, -0.0244,  ..., -0.0537, -0.0037,  0.0378],
        [ 0.0408, -0.0261,  0.0214,  ..., -0.0776, -0.0013, -0.0064],
        ...,
        [-0.0272, -0.0254,  0.0016,  ..., -0.0479,  0.0339,  0.0535],
        [-0.0109,  0.0195,  0.0005,  ..., -0.0520, -0.0298,  0.0072],
        [-0.0186, -0.0315, -0.0067,  ..., -0.0737,  0.0508,  0.0366]],
       dtype=torch.bfloat16)

In [10]:
tensor['text_emb'][0]

tensor([ 0.0037, -0.0552, -0.0244,  ..., -0.0537, -0.0037,  0.0378],
       dtype=torch.bfloat16)

In [11]:
tensor['text_emb'][1]

tensor([ 0.0037, -0.0552, -0.0244,  ..., -0.0537, -0.0037,  0.0378],
       dtype=torch.bfloat16)

In [12]:
tensor['text_emb'][2]

tensor([ 0.0408, -0.0261,  0.0214,  ..., -0.0776, -0.0013, -0.0064],
       dtype=torch.bfloat16)

In [13]:
tensor['text_emb'][0] == tensor['text_emb'][1]

tensor([True, True, True,  ..., True, True, True])

In [14]:
for i in (tensor['text_emb'][0] == tensor['text_emb'][1]):
    if i == True:
        continue
    else:
        print(False)
        break

In [15]:
torch.sum(tensor['text_emb'][0])

tensor(0.5859, dtype=torch.bfloat16)

In [16]:
torch.norm(torch.nn.functional.normalize(tensor['text_emb'][0],dim=-1))

tensor(1., dtype=torch.bfloat16)